# Introduction

This material is an introduction to using machine-learning for portfolio management and trading. Given the centrality of programming in hedge funds today, the concepts are exposed using only `jupyter` notebooks in `python`. Moreover, we leverage the `scikit-learn` package (also known as `sklearn`) to illustrate how machine-learning is used in practice in this context.

## Outline

Outline:

1. **Backtesting:** Markowitz portfolio optimisation, industry momentum

1. **Risk:** risk-model shrinkage, non-normal returns

1. **Linear estimators:** pipelines Ridge, Lasso, feature engineering

1. **Boosted trees and neural nets:** Lightgbm, multi-layer perceptron

1. **Factors:** Value, Momentum, Style Analysis 

1. **Overfitting:** hyperparameter search, validation strategy

1. **Transaction cost:** turnover, leverage, portfolio optimisation with constraints

1. **Mean reversion:** profitability of liquidity provision, survival-free sample 

1. **Event trading:** macroeconomic events 

1. **Text:** document clustering, tfidf

1. **Sentiment:** rule-based or learning based sentiment

1. **Pretrained language models:** fine-tuning language models on GPUs 

We are interested in how quantitative hedge funds operate in practice. Today, quantitative hedge funds are essentially *consumers* of data -- they ingest all sorts of datasets and extract information used to buy or sell systematically securities. Researchers and portfolio managers are deeply involved in the process of `data ingestion` and `information extraction`, but they do not directly decide which securities are bought or sold -- instead algorithms do.

Because these processes of `data ingestion` and `information extraction` are so central to quantitative hedge fund operations, they have become `software companies` -- a lot of the intellectual property (IP) of hedge funds is embedded in the code they write. And in that sense, hedge funds are not so different from other data-science based technology companies. (And in fact the hiring has become very similar, with a lot of interest in profiles out of Computer Science, Machine-Learning, Data engeeniring, Statistics, etc).

In the rest of this section, we first describe how new datasets have become available and what it implies for `Machine-learning` research and more particularly, for quantitative hedge funds. We then introduce the structure of this course. In particular, given this course will focus on code (that is, `python` code), it is written entirely in `jupyter` notebooks.

## Trends

### Data deluge

In [ ]:
#hide
from IPython.display import display, Image

There are now sensors everywhere in the physical world and most of the online interactions are tracked -- leading a `data deluge" (e.g. see [Mary Meeker (2018)](http://www.kpcb.com/internet-trends) on internet trends or [Lori Lewis (2022)](https://www.allaccess.com/merge/archive/31294/infographic-what-happens-in-an-internet-minute)). 

In [ ]:
#hide
#sidebyside
display(Image('images/sensors.PNG',width=500), Image('images/internet_minute_2020.png', width=400))

### Data scouting

>  Alternative data = web + credit card transactions + geolocalisation + satellite imaging


Two examples on how data is transforming hedgd funds in the Financial Times and Bloomberg: [FT (08/28/2017)](https://www.ft.com/content/d86ad460-8802-11e7-bf50-e1c239b45787) and [Bloomberg (06/15/2019)](https://www.bloomberg.com/news/articles/2019-05-15/quants-think-like-amateurs-in-the-world-s-wildest-stock-market).

In [ ]:
#hide 
display(Image("images/ftalternativedata.png", width=700))

In [ ]:
#hide 
display(Image("images/bloomberg_investing_in_china.png", width=600))

### ML research

ML research has become a race with new ideas coming out with an increasing speed -- e.g. as illustrated by the number of papers published on the scientific paper repository arxiv.com ([Jeff Dean (06/02/2019)](https://twitter.com/JeffDean/status/1135114657344237568)). More precisely, as [Francois Chollet (04/03/2019)](https://twitter.com/fchollet/status/1113476428249464833) points out, the issue is how to process (ie. test empirically) these new ideas -- as fast as possible to gain a competitive edge. 

In [ ]:
#hide 
#sidebyside 
display(Image("images/mlarxiv2.png", width=600), Image("images/kaggle_iterations2.png", width=500))

The success of deep-learning depends on: i) model "capacity", ii) computational power, iii) dataset  size. [Sun, Shrivastava, Singh, Gupta (2017)](https://arxiv.org/abs/1707.02968) note that the size of the largest dataset has remained somewhat constant over the last few years. 

In [ ]:
#hide 
#sidebyside 
display(Image('images/unreasonableEffectiveness.PNG',width=300), Image('images/unreasonableEffectiveness2.PNG', width=300))

A particular success of deep-learning has been on Natural Language Processing (NLP) and there too,  the size of the largest models has increased dramatically. [Victor Sanh, Lysandre Debut, Julien Chaumond, Thomas Wolf (2019)](https://arxiv.org/pdf/1910.01108.pdf).

In [ ]:
#hide 
display(Image("images/nlp_model_size.png", width=800))

## Quant workflow

The graph below shows the typical data workflow of a quantitative fund: 

- raw data is acquired generally by a Data team and possibly transformed into usable features. 
- from these features, predictors of asset returns are constructed 
- given a single predictor (or a set of many predictors), porfolios constructed: these portfolios represent the ideal positions of a fund given the asset forecasts, but also risk forecasts (and possibly, transaction cost forecasts). 
- when these ideal positions change from one day to the next (because the underlying data has been updated), the difference in positions initiate trades that are then executed on asset markets or with brokers. 


In [ ]:
#hide 
display(Image("images/quant_workflow.png", width=800))

## MLOps

MLOps (machine learning operations) represents a set of practices for the deployment of ML models in production. For quant hedge funds, there are two main concepts that we describe here:  

- pipelines 

- backtests 

### Pipelines 

Pipeline: 

> A machine-learning pipeline is an end-to-end description of the automated flow of data from raw inputs to a desired output. Each step represents a transformation of the data, possibly with a fitted model. 


The diagram below illustrates a pipeline for a quant fund. The end point (to the right of the diagram) are the `holdings` in a set of traded securities -- and combined with the returns on these securities, the pnl of a given strategy can be computed. The entry point (to the left of the diagram) are `features`. A set of `transformations` (pre-determined in the `pipeline`) are applied to these `features` to produce the desired `holdings`. Some `transformations` in the `pipeline` are "fixed" while others depend on `fitted models` (e.g. a ML predictor of returns or a risk model). 

In the diagram, we emphasize the timing of these different objects: 

- for a pnl at time $t$, the `features` and `target` include only information up to $t-1$ so that the holdings are known in $t-1$ and can accrue returns over the period $t$. 

The following equation summarizes this point: 

$$ pnl_t = holdings_{t-1} \times returns_t. $$

In [ ]:
#hide 
display(Image("images/ml4pmt_pipeline3.png", width=500))

### Scikit-learn

The following notebooks and notes are largely based on  `scikit-learn`. `scikit-learn` is an extremely powerful (and widely used) package for machine-learning. In particular, it provides a "grammar" for pipelines  where each transformation or estimator class has the `fit`/`transform`/`predict` functions with arguments as (`X`, `y`) where `X` represents the  features and `y`, targets. 

### Backtesting

A look-ahead bias occurs when data dated at $t$ includes information only available after $t$; in contrast, point-in-time data ensures that data dated at t is based on only information up to date t. A backtest is a method to simulate a strategy using point-in-time historical data and evaluate its profitability. 

In order to illustrate how to use pipelines à la `scikit-learn` for quantitative portfolio management, we introduce a thin layer of functions -- in particular, the `backtester` class. This class allow to run a rolling window simulation so that only information up to date $t-1$ is used to determined the holdings at that date. 

In [ ]:
#hide 
display(Image("images/layered_api.png", width=300))